In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import matplotlib.pyplot as plt
import plotly.graph_objs as go
import networkx as nx
import pandas as pd
import community
import plotly
import re

In [2]:
f = open("Freeman’s EIES dataset.csv")

In [3]:
G = nx.Graph()

In [4]:
# add edges to graph
lines = f.readlines()

for line in lines:
    nodes = line.split()
    G.add_edge(nodes[0], nodes[1], weight=int(nodes[2]))

In [5]:
G.number_of_nodes()

34

In [6]:
G.number_of_edges()

415

In [7]:
# generate layout of the graph
coordinates = nx.spring_layout(G, center=[0.5, 0.5], scale=10, weight = 'weight')

In [8]:
name_f = open("name.txt")

name_alias = []
            
for l in name_f.readlines():
    names = l.split("\"")
    index = -1
    while index < len(names) - 1:
        index += 1
        if (re.match("[a-zA-Z]+\s[a-zA-Z]+", names[index])):
            name_alias.append(names[index])

In [9]:
i = 0
for node, pos in coordinates.items():
    G.add_node(node, pos=pos, name=name_alias[i])
    i += 1

In [10]:
edge_trace = go.Scatter(
    x=[],
    y=[],
    name='edges',
    line=dict(width=0.5, color='#C5C6B6'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():    
    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

In [11]:
affiliation = ['sociology', 'anthropology', 'mathematics/statistics', 'others']

In [12]:
node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='Portland',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(
            width = 2,
            color = '#FFFFFF')))

for node in G.nodes():
    x, y = G.node[node]['pos']
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])

In [13]:
# add number of connections
connections = []
for node, adjacencies in enumerate(G.adjacency()):
    node_trace['marker']['color'] += tuple([len(adjacencies[1])])
    node_info = 'connections=' + str(len(adjacencies[1]))
    connections += tuple([node_info])

alias = list(connections)

# add node name
i = 0
for node in G.nodes():
    alias[i] = alias[i] + ', name=' + G.node[node]['name']
    i += 1

# add affiliation
i = 0
affi_coloration = []
for string in open("affiliation.txt").readlines():
    groups = string.split();
    for grp in groups:
        alias[i] = alias[i] + ', affiliation=' + affiliation[int(grp) - 1]
        affi_coloration.append(int(grp))
        i += 1
        
node_trace['text'] = tuple(alias)

In [14]:
# display the network graph with number of connections
fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                title='<br>Freeman’s EIES networks',
                titlefont=dict(size=14),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="nodes coloration based on number of connections",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

iplot(fig)

In [15]:
sociology = []
anthropology = []
math = []
others = []

trace_soc = go.Scatter(
    x = [],
    y = [],
    text = [],
    hoverinfo = 'text',
    name = 'sociology',
    mode = 'markers',
    marker = dict(
        color = '#E53A40',
        size = 10,
        line = dict(width = 2)))

trace_anth = go.Scatter(
    x = [],
    y = [],
    text = [],
    hoverinfo = 'text',
    name = 'anthropology',
    mode = 'markers',
    marker = dict(
        color = '#30A9DE',
        size = 10,
        line = dict(width = 2)))

trace_math = go.Scatter(
    x = [],
    y = [],
    text = [],
    hoverinfo = 'text',
    name = 'mathematics/statistics',
    mode = 'markers',
    marker = dict(
        color = '#EFDC05',
        size = 10,
        line = dict(width = 2)))

trace_others = go.Scatter(
    x = [],
    y = [],
    text = [],
    hoverinfo = 'text',
    name = 'others',
    mode = 'markers',
    marker = dict(
        color = '#8CD790',
        size = 10,
        line = dict(width = 2)))

i = 0
for node in G.nodes():
    x, y = G.node[node]['pos']
    # sociology
    if affi_coloration[i] == 1:
        trace_soc['x'] += tuple([x])
        trace_soc['y'] += tuple([y])
        sociology.append(name_alias[i])
    # anthropology
    elif affi_coloration[i] == 2:
        trace_anth['x'] += tuple([x])
        trace_anth['y'] += tuple([y])
        anthropology.append(name_alias[i])
    # mathematics/statistics
    elif affi_coloration[i] == 3:
        trace_math['x'] += tuple([x])
        trace_math['y'] += tuple([y])
        math.append(name_alias[i])
    # others
    elif affi_coloration[i] == 4:
        trace_others['x'] += tuple([x])
        trace_others['y'] += tuple([y])
        others.append(name_alias[i])

    i += 1
    
trace_soc['text'] = tuple(sociology)
trace_anth['text'] = tuple(anthropology)
trace_math['text'] = tuple(math)
trace_others['text'] = tuple(others)

In [16]:
# display the network graph with affiliation
affi_fig = go.Figure(data=[edge_trace, trace_soc, trace_anth, trace_math, trace_others],
                layout=go.Layout(
                title='<br>Freeman’s EIES networks',
                titlefont=dict(size=14),
                showlegend=True,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="nodes coloration based on affiliation",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

iplot(affi_fig)

In [17]:
close_frd_trace = go.Scatter(
    x = [],
    y = [],
    name = 'close friends (edge weight = 3, 4)',
    line = dict(width=2, color='#000000'),
    hoverinfo = 'none',
    mode = 'lines')

not_close_frd_trace = go.Scatter(
    x = [],
    y = [],
    name = 'not close friends (edge weight = 0, 1, 2)',
    line = dict(width=0.5, color='#C5C6B6'),
    hoverinfo = 'none',
    mode = 'lines')
    
for edge in G.edges():
    weight = G[edge[0]][edge[1]]['weight']
    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    if weight == 3 or weight == 4:
        close_frd_trace['x'] += tuple([x0, x1, None])
        close_frd_trace['y'] += tuple([y0, y1, None])
    else:
        not_close_frd_trace['x'] += tuple([x0, x1, None])
        not_close_frd_trace['y'] += tuple([y0, y1, None])
        
# display the network graph with close friend relationship in highlight
close_frd_fig = go.Figure(
                data=[close_frd_trace, not_close_frd_trace,
                    trace_soc, trace_anth, trace_math, trace_others],
                layout=go.Layout(
                title='<br>Freeman’s EIES networks',
                titlefont=dict(size=14),
                showlegend=True,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="edges coloration based on whether or not edge is close friend relationship (edge weight = 3, 4)",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

iplot(close_frd_fig)

In [18]:
partitions = community.best_partition(G)

part0_trace = go.Scatter(
    x = [],
    y = [],
    text = [],
    hoverinfo = 'text',
    name = 'partition A',
    mode = 'markers',
    marker = dict(
        color = '#6AAFE6',
        size = 10,
        line = dict(width = 2)))

part1_trace = go.Scatter(
    x = [],
    y = [],
    text = [],
    hoverinfo = 'text',
    name = 'partition B',
    mode = 'markers',
    marker = dict(
        color = '#f9c00c',
        size = 10,
        line = dict(width = 2)))

part0_text = []
part1_text = []

i = 0
for node in partitions.keys():
    x, y = G.node[node]['pos']
    if (partitions[node] == 0):
        part0_trace['x'] += tuple([x])
        part0_trace['y'] += tuple([y])
        part0_text.append(alias[i])
    else:
        part1_trace['x'] += tuple([x])
        part1_trace['y'] += tuple([y])
        part1_text.append(alias[i])
    i += 1

part0_trace['text'] = tuple(part0_text)
part1_trace['text'] = tuple(part1_text)    
    
part_fig = go.Figure(
                data=[edge_trace, part0_trace, part1_trace],
                layout=go.Layout(
                title='<br>Freeman’s EIES networks',
                titlefont=dict(size=14),
                showlegend=True,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="nodes coloration based on partition",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

iplot(part_fig)